In [10]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../driftpy/src/')

import pandas as pd 
import numpy as np 

pd.options.plotting.backend = "plotly"

from driftpy.constants.config import configs
from anchorpy import Provider, Wallet
from solana.keypair import Keypair
from solana.rpc.async_api import AsyncClient
from driftpy.clearing_house import ClearingHouse
from driftpy.accounts import *
from driftpy.constants.numeric_constants import * 

In [11]:
def human_amm_df(amm: AMM):
    bool_fields = [ 'last_oracle_valid']
    enum_fields = ['oracle_source']
    pure_fields = ['last_update_slot', 'long_intensity_count', 'short_intensity_count', 
    'curve_update_intensity', 'amm_jit_intensity'
    ]
    reserve_fields = [
        'base_asset_reserve', 'quote_asset_reserve', 'min_base_asset_reserve', 'max_base_asset_reserve', 'sqrt_k'
        'ask_base_asset_reserve', 'ask_quote_asset_reserve', 'bid_base_asset_reserve', 'bid_quote_asset_reserve'
        'terminal_quote_asset_reserve', 'base_asset_amount_long', 'base_asset_amount_short', 'base_asset_amount_with_amm', 'base_asset_amount_with_unsettled_lp',
        'user_lp_shares'
        ]
    pct_fields = ['long_spread', 'short_spread', 'concentration_coef',]
    funding_fields = ['cumulative_funding_rate_long', 'cumulative_funding_rate_short', 'last_funding_rate', 'last_funding_rate_long', 'last_funding_rate_short', 'last24h_avg_funding_rate']
    quote_asset_fields = ['total_fee', 'total_mm_fee', 'total_exchange_fee', 'total_fee_minus_distributions',
    'total_fee_withdrawn', 'total_liquidation_fee', 'cumulative_social_loss', 'net_revenue_since_last_funding',
    'quote_asset_amount_long', 'quote_asset_amount_short', 'quote_entry_amount_long', 'quote_entry_amount_short',
    'volume24h', 'long_intensity_volume', 'short_intensity_volume',]
    time_fields = ['last_mark_price_twap_ts', 'last_oracle_price_twap_ts']
    duration_fields = ['lp_cooldown_time']
    px_fields = ['last_bid_price_twap', 'last_ask_price_twap', 'last_mark_price_twap', 'last_mark_price_twap5min',
    'peg_multiplier',
    'mark_std']
    pool_fields = ['fee_pool']


def human_market_df(market: PerpMarket):
    enum_fields = ['status', 'contract_tier', '']
    pure_fields = ['number_of_users', 'market_index', 'next_curve_record_id', 'next_fill_record_id', 'next_funding_rate_record_id']
    pct_fields = ['imf_factor', 'unrealized_pnl_imf_factor', 'liquidator_fee', 'if_liquidation_fee',
    'margin_ratio_initial', 'margin_ratio_maintenance']
    px_fields = ['expiry_price', ]
    time_fields = ['last_trade_ts', 'expiry_ts']
    pool_fields = ['pnl_pool']

In [12]:
config = configs['devnet']
url = 'https://api.devnet.solana.com'
import json
print(config)
# random key 
with open(os.path.expanduser('~/.config/solana/.json'), 'r') as f: secret = json.load(f) 
kp = Keypair.from_secret_key(bytes(secret))
print('pk:', kp.public_key)
wallet = Wallet(kp)
connection = AsyncClient(url)
provider = Provider(connection, wallet)
ch = ClearingHouse.from_config(config, provider)

Config(env='devnet', pyth_oracle_mapping_address=BmA9Z6FjioHJPpjT39QazZyhDRUdZy2ezwx4GiDdE2u2, clearing_house_program_id=DUZwKJKAk2C9S88BYvQzck1M1i5hySQjxB4zW6tJ29Nw, usdc_mint_address=8zGuJQqwhZafTah7Uc7Z4tXRnguqkn5KLFAP8oV6PHe2, markets=[Market(symbol='SOL-PERP', base_asset_symbol='SOL', market_index=0, pyth_oracle=J83w4HKfqxwcq3BEMMkPFSppX3gqekLyLJBexebFVkix), Market(symbol='BTC-PERP', base_asset_symbol='BTC', market_index=1, pyth_oracle=HovQMDrbAgAYPCmHVSrezcSmkMtXSSUsLDFANExrZh2J), Market(symbol='ETH-PERP', base_asset_symbol='ETH', market_index=2, pyth_oracle=EdVCmQ9FSPcVe5YySXDPCRmc8aDQLKJ9xvYBMZPie1Vw)], banks=[Bank(symbol='USDC', bank_index=0, oracle=11111111111111111111111111111111, oracle_source=<class 'sumtypes.QUOTE_ASSET'>, mint=8zGuJQqwhZafTah7Uc7Z4tXRnguqkn5KLFAP8oV6PHe2), Bank(symbol='SOL', bank_index=1, oracle=J83w4HKfqxwcq3BEMMkPFSppX3gqekLyLJBexebFVkix, oracle_source=<class 'sumtypes.PYTH'>, mint=So11111111111111111111111111111111111111112), Bank(symbol='BTC', bank_i

In [13]:
state = await get_state_account(ch.program)
state

State(admin=E7iAhFMa9KvwhJtaPpoqJE4wPZb16zj2Az7PM9YdWK15, whitelist_mint=11111111111111111111111111111111, discount_mint=11111111111111111111111111111111, signer=FrUhDdSssSknnCfudsWj2NQhdUxYFKV8T9DS8ogMnd8W, srm_vault=11111111111111111111111111111111, perp_fee_structure=FeeStructure(fee_tiers=ListContainer([FeeTier(fee_numerator=100, fee_denominator=100000, maker_rebate_numerator=20, maker_rebate_denominator=100000, referrer_reward_numerator=15, referrer_reward_denominator=100, referee_fee_numerator=5, referee_fee_denominator=100), FeeTier(fee_numerator=80, fee_denominator=100000, maker_rebate_numerator=20, maker_rebate_denominator=100000, referrer_reward_numerator=15, referrer_reward_denominator=100, referee_fee_numerator=5, referee_fee_denominator=100), FeeTier(fee_numerator=60, fee_denominator=100000, maker_rebate_numerator=20, maker_rebate_denominator=100000, referrer_reward_numerator=15, referrer_reward_denominator=100, referee_fee_numerator=5, referee_fee_denominator=100), FeeTie

In [14]:
market = await get_perp_market_account(ch.program, 0)